# General

Things to fix or add:

Fourier transform of the Hahn Echo

Check Hahn Echo in qudi

Spin density in XY8

Add exponential decay in XY8 fitting

confocal imaging, ODMR

## Library import

Importing libraries and qudi_analysis where main constants and functions are defined.

In data import second argument is used to extract data with an alternating signal (number of columns).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
from scipy.optimize import curve_fit
from scipy.fft import rfft, rfftfreq
import plotly.express as px
import os

import qudi_analysis_functions as qa

# XY8
## Experimental data and estimation

### Data import

Uploading qudi .dat format data files for the analysis. Change directory and filenames of the data.

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\400 G\Rabi")
# Rabi
filename = r"20220920-1218-09_Sample 19_LiF_NV11_Rabi_384G_pulsed_measurement.dat"
Rabi_time, Rabi_signal = qa.qudi_reader(filename , 2)


os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\400 G\XY8")
# XY8-N; Region of interest
filename=r"20220921-1722-04_Sample 19_LiF_NV11_XY8_500_384G_pulsed_measurement.dat"
XY8_roi_time, XY8_roi_signal, XY8_roi_signal_alt, XY8_roi_delta = qa.qudi_reader(filename, 3)

# # XY8-N; full
# filename = r"20220913-0807-59_Sample 19_LiF_NV1_XY8_F_384G_pulsed_measurement.dat"
# XY8_time, XY8_signal, XY8_signal_alt, XY8_delta = qa.qudi_reader(filename, 3)


### Variables for the analysis

Values to change:

Sample_name: Uneder this name data files will be saved.

N: Number of XY8 blocks

nor: Number of resonance peaks.

ODMR_left & ODMR_right: ODMR dips from the qudi fit.

amplitude, freq, offset: Rabi fitting values.

gamma & gamma2 (optionally): Measured isotop(s)
reduced gyromagnetic ratio(s).

gamma_name & gamma2_name (optionally): Measured isotop(s) reduced
gyromagnetic ratio(s).

In [ ]:
Sample_name = 'Sample 19 LiF NV11 7Li (2)'

# Number of XY8 blocks
N = 4

# Number of resonances
nor = 2 # 1 or 2

# Magnetic field (use values from the qudi ODMR fitting)
ODMR_left = 1796 * 1e6    #Hz
ODMR_right = 3950.49 * 1e6    #Hz
B_ext = (ODMR_right - ODMR_left) / (2 * qa.gamma_NV)    #G
# B_ext = 490.52   # G

# Rabi fitting (use values from the qudi .png file)
amplitude = 0.2    # a.u.
freq = 17    # MHz
offset = 0.9    # a.u.
phase = 0    # rad

# Select isotopes 
gamma = qa.gamma_7Li    # Hz/G
gamma_name = '7Li'
if nor == 2: # Isotope with lower gyromagnetic ratio
    gamma2 = qa.gamma_7Li    # Hz/G
    gamma2_name = '7Li'
else:
    gamma2 = gamma
    gamma2_name = gamma_name

### Larmor frequency and tau spacing

qa.Estimation will print calulated gyromagnetic ratio, Larmor frequency and an expected value of tau spacing for the XY8 measurements.

In [ ]:
if nor == 2: 
    tau, Larm_freq, tau2, Larm_freq2 = qa.Estimation(nor, B_ext, gamma, gamma2, gamma_name, gamma2_name)
else:
    tau, Larm_freq = qa.Estimation(nor, B_ext, gamma, gamma2, gamma_name, gamma2_name)

## Data analysis

### Normalization and spectral density

Values to change:

skip_points_start: If XY8_roi_time starts from 0 or really small values, it is better to start from higher tau values for the power spectral density calculation.

skip_points_end: To remove some noise.

error_fix: If there is an error during spectral density calculation.

In [ ]:
skip_points_start = 0
skip_points_end = 0
error_fix = 0 # increase with 0.1 step

# Rabi fitting
Rabi_p0 = np.array([amplitude, freq, phase, offset]) 
Rabi_popt, Rabi_pcov = curve_fit(qa.cos_func, Rabi_time, Rabi_signal, Rabi_p0)

# XY8 normalization
level = ((XY8_roi_signal + XY8_roi_signal_alt)/2).mean()
XY8_norm = qa.norm(XY8_roi_signal, qa.cos_func(Rabi_time, *Rabi_popt), level)
XY8_norm_alt = qa.norm(XY8_roi_signal_alt, qa.cos_func(Rabi_time, *Rabi_popt), level)

# XY8 mean delta of 2 signals
XY8_delta = (XY8_norm + (1 - XY8_norm_alt)) / 2

# Power spectral density calculation
skip_points_end = len(XY8_roi_time) - skip_points_end
if skip_points_start != 0 or  skip_points_end != 0:
    # Calculates frequency Nu for given values of tau, kHz
    XY8_frequency = 1000 / (2 * XY8_roi_time[skip_points_start:skip_points_end])
    # In original script instead of np.max(Rabi_signal) + error_fix was just 1 which gives errors if signals are overlapping
    Spectral_dens = qa.deconv(8 * N, np.max(Rabi_signal) + error_fix - XY8_delta[skip_points_start:skip_points_end], XY8_roi_time[skip_points_start:skip_points_end])
else:
    # Calculates frequency Nu for given values of tau, kHz
    XY8_frequency = 1000 / (2 * XY8_roi_time)    
    Spectral_dens = qa.deconv(8 * N, np.max(Rabi_signal) + error_fix - XY8_delta, XY8_roi_time)

### Plotting Rabi, XY8 signal and spectral density

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (20,13))

axs[0,0].plot(Rabi_time, Rabi_signal)
axs[0,0].set(xlabel = 'Tau (ns)', ylabel = 'Florescence (a.u.)', title = 'NV Rabi')
axs[0,0].plot(Rabi_time, qa.cos_func(Rabi_time, *Rabi_popt), 'r--', label='fit:\nampl=%5.3f \nfreq=%5.3f MHz \nphase=%5.3f \noffset=%5.3f' % tuple(Rabi_popt))
axs[0,0].legend()

axs[0,1].plot(XY8_roi_time, XY8_roi_signal, label = 'pi/2')
axs[0,1].plot(XY8_roi_time, XY8_roi_signal_alt, label = '3pi/2')
axs[0,1].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'NV XY8-' + str(N))
axs[0,1].plot(XY8_roi_time, qa.cos_func(XY8_roi_time, *Rabi_popt), 'r--', label='Rabi')
axs[0,1].legend()

axs[1,0].plot(XY8_roi_time, XY8_norm, label = 'pi/2')
axs[1,0].plot(XY8_roi_time, XY8_norm_alt, label = '3pi/2')
axs[1,0].plot(XY8_roi_time, XY8_delta, '--', label = 'Mean delta')
axs[1,0].set(xlabel = 'Tau (us)', ylabel = 'Florescence (norm.)', title = 'NV XY8-'+ str(N) + ' normalized data')
# Expected sensing spacing
axs[1,0].axvline(x = tau, color = 'grey', linestyle = '--')
axs[1,0].text(tau, np.max(XY8_norm_alt), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[1,0].axvline(x = tau2, color = 'olive', linestyle = '--')
    axs[1,0].text(tau2, np.max(XY8_norm_alt) - 0.1, gamma2_name, color = 'olive', fontsize = 15)
axs[1,0].legend()

axs[1,1].plot(XY8_frequency, Spectral_dens, 'g', label = 'Mean delta')
axs[1,1].set(xlabel = 'Frequency (kHz)', ylabel = 'Power spectral density (kHz)', title = 'NV spectral density')
# Expected Larmor frequencies
axs[1,1].axvline(x = Larm_freq, color = 'grey', linestyle = '--')
axs[1,1].text(Larm_freq, np.max(Spectral_dens), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[1,1].axvline(x = Larm_freq2, color = 'olive', linestyle = '--')
    axs[1,1].text(Larm_freq2, np.max(Spectral_dens) - 50, gamma2_name, color = 'olive', fontsize = 15)
axs[1,1].legend()

plt.savefig(Sample_name + " spectral density.png")

### Interactive plot

Change signal and time accordingly. You can use this plot to determine peak frequencies, amplitudes and FWHM for the fitting.

In [ ]:
signal = Spectral_dens
time = XY8_frequency


figexp = px.line(x = time, y = signal, title="FT")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text = "Frequency (MHz)")
figexp.update_yaxes(title_text = "Amplitude(arb. units)")
figexp.show(config = config)

### Lorentzian fitting

Values to change:

amplitude, center, sigma, baseline: Lorentzian fitting values, use interactive plot to correct values.

center2: Depending on the highest peak position you need to either sum or substract.

In [ ]:
amplitude = np.max(Spectral_dens)
center = XY8_frequency[np.argmax(Spectral_dens)]
sigma = 30
baseline = 40

if nor == 2:
    center2 = center + (Larm_freq - Larm_freq2) # OR = center - (Larm_freq - Larm_freq2)
    Lorentzian_p0 = np.array([amplitude, center, sigma, amplitude, center2, sigma, baseline])
#     Lorentzian_p0 = np.array([117, 675, 14, 85, 770, sigma, baseline])
    Spectrum_fit, pcov = curve_fit(qa.double_Lorentzian_func, XY8_frequency, Spectral_dens, Lorentzian_p0)
else:
    Lorentzian_p0 = np.array([amplitude, center, sigma, baseline])
#     Lorentzian_p0 = np.array([54, 770, 20, 0])
    Spectrum_fit, pcov = curve_fit(qa.Lorentzian_func, XY8_frequency, Spectral_dens, Lorentzian_p0)
    
print(Lorentzian_p0)

### Spectral density plotting

In [ ]:
fig, axs = plt.subplots(figsize = (18,10))
axs.plot(XY8_frequency, Spectral_dens, 'o',label = 'Delta')
if nor == 2:
    axs.plot(XY8_frequency, qa.double_Lorentzian_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \namplitude2=%5.3f \ncenter2=%5.3f  \nsigma2=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
else:
    axs.plot(XY8_frequency, qa.Lorentzian_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
axs.legend()
axs.set(xlabel = 'Frequency (kHz)', ylabel = 'Power spectral density (kHz)', title = 'Noise spectral density')

plt.savefig(Sample_name + " spectrum.png")

### Expreimental Larmor frequency and magnetic field

Prints difference of experimental values with the theoretical calculations.

In [ ]:
# Experimental Larmor frequency and magnetic field
peak_freq = Spectrum_fit[1]
B_calc = peak_freq * 1e3 / gamma2

# Checking that fitted peaks relative positions are same as theoretical
if nor == 2:
    peak_freq2 = Spectrum_fit[4]
    B_calc2 = peak_freq2 * 1e3 / gamma2
    if peak_freq < peak_freq2:
        check = 1
        peak_freq, peak_freq2 = peak_freq2, peak_freq # Comment out if frequencies are correct from the beggining
        B_calc, B_calc2 = B_calc2, B_calc
    else:
        check = 0
    print(gamma_name + ' Larmor frequency from the experimental data:', round(peak_freq,2), 'kHz')
    print(gamma_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq - Larm_freq,2), 'kHz')
    print(gamma_name + ' magnetic field from the experimental data:', round(B_calc,2), 'G')
    print(gamma_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc - B_ext,2), 'G')
    print('\nFor the second signal:')
    print(gamma2_name + ' Larmor frequency from the experimental data:', round(peak_freq2,2), 'kHz')
    print(gamma2_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq2 - Larm_freq2,2), 'kHz')
    print(gamma2_name + ' magnetic field from the experimental data:', round(B_calc2,2), 'G')
    print(gamma2_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc2 - B_ext,2), 'G')
else:
    print(gamma_name + ' Larmor frequency from the experimental data:', round(peak_freq,2), 'kHz')
    print(gamma_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq - Larm_freq,2), 'kHz')
    print(gamma_name + ' magnetic field from the experimental data:', round(B_calc,2), 'G')
    print(gamma_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc - B_ext,2), 'G')

### NMR signal

Calculates Lorentzian linewidth and magnetic field fluctuations.

In [ ]:
if nor == 2:
    # Getting the integral of the magnetic noise spectrum with substracted offset
    spectrum_area = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[0:3], Spectrum_fit[6]) - Spectrum_fit[6], x = np.sort(XY8_frequency))/ 1e6    # MHz^2
    spectrum_area2 = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[3:]) - Spectrum_fit[6], x = np.sort(XY8_frequency))/ 1e6     #MHz^2
    # Lorentzian linewidth with the substracted offset
    linewidth = 2 * np.abs(Spectrum_fit[2])
    linewidth2 = 2 * np.abs(Spectrum_fit[5])
    if check == 1:
        spectrum_area, spectrum_area2 = spectrum_area2, spectrum_area
        linewidth, linewidth2 = linewidth2, linewidth
    print(gamma_name + ' Lorentzian linewidth:', round(linewidth,2), 'kHz')
    print(gamma2_name + ' Lorentzian linewidth:', round(linewidth2,2), 'kHz')
else:
    # Getting the integral of the magnetic noise spectrum with substracted offset
    spectrum_area = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[0:3], Spectrum_fit[3]) - Spectrum_fit[3], x = np.sort(XY8_frequency))/ 1e6    # MHz^2
    # Lorentzian linewidth with the substracted offset
    linewidth = 2 * np.abs(Spectrum_fit[2])
    print(gamma_name + ' Lorentzian linewidth:', round(linewidth,2), 'kHz')
    
# The magnetic field fluctuations (NMR magnetic signal)
B1 = (2 * spectrum_area / qa.gamma_e**2)**0.5    # T
print(gamma_name + ' B1 =', round(B1 * 1e6,2), 'uT')
if nor == 2:
    B2 = (2 * spectrum_area2 / qa.gamma_e**2)**0.5    # T
    print(gamma2_name + ' B2 =', round(B2 * 1e6,2), 'uT')

### NV center depth

Calculating the depth of the NV in the case of the hydrogen spins in immersion oil.

In [ ]:
depth1 = qa.depth(B1, gamma * 1e4, qa.rho_oil)
print('depth1 =', depth1, 'nm')
if nor == 2:
    depth2 = qa.depth(B2, gamma * 1e4, qa.rho_oil)
    mean_depth = (depth1 + depth2) / 2
    print('depth2 =', depth2, 'nm')
    print('mean depth =', mean_depth, 'nm')

In [ ]:
## calcuate the spin density for given depth:

In [ ]:

# d1 = 3.7027003282742967
# d2 = 3.7027003282742967
# rho1 = dc.calculate_density_simple(mu_0, h, B1, gamma_p, d1)
# rho2 = dc.calculate_density_simple(mu_0, h, B2, gamma_p, d2)
# print('rho1 =', rho1, 'nm^-3')
# print('rho2 =', rho2, 'nm^-3')

# Hahn Echo

## Experimental data and estimation

### Data import

qudi for Hahn Echo measurements creates, plots and saves data with tau spacing between pulses instead of tau/2.

In [ ]:
## Check again data saving!!

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Small magnetic field\Hahn Echo")

# Hahn Echo
filename = r"20220909-1518-17_Sample 19_LiF_NV1_Hahn Echo_pulsed_measurement.dat"
T2_time, T2_signal, T2_signal_alt, T2_delta = qa.qudi_reader(filename, 3)
# qudi for Hahn Echo measurements creates, plots and saves data with tau spacing between pulses instead of tau/2.
# Therefore T2_time should be two times larger.
# T2_time = 2 * T2_time

os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Small magnetic field\Rabi")
# Rabi
filename = r"20220909-1354-06_Sample 19_LiF_NV1_Rabi_pulsed_measurement.dat"
Rabi_time, Rabi_signal = qa.qudi_reader(filename , 2)

### Variables for the analysis

In [ ]:
Sample_name = 'Sample'

# Rabi fitting (use values from the qudi .png file)
amplitude = 0.2    # a.u.
freq = 22    # MHz
offset = 0.9    # a.u.
phase = 0    # rad

## Data analysis

### Coherence time

Transforms data to correct tau spacing and calculates T2 time, fits the exponential decay.

In [ ]:
## Check how qudi saves the data!

In [ ]:
T2_popt, T2_pcov = curve_fit(qa.T2_func, T2_time, T2_delta)
print('Coherence time T2 =', round(T2_popt[1],2), 'us')

#Rabi offset
Rabi_p0 = np.array([amplitude, freq, phase, offset]) 
Rabi_popt, Rabi_pcov = curve_fit(qa.cos_func, Rabi_time, Rabi_signal, Rabi_p0)

fig, axs = plt.subplots(1, 2, figsize = (18,5))
axs[0].plot(T2_time, T2_signal, label = 'pi/2')
axs[0].plot(T2_time, T2_signal_alt, label = '3pi/2')
axs[0].plot(T2_time, T2_delta, '--', label = 'Mean delta')
axs[0].axhline(y = Rabi_popt[3], color = 'grey', linestyle = ':')
axs[0].text(0, Rabi_popt[3], 'Rabi offset', color = 'grey', fontsize = 13)
axs[0].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'NV Hahn Echo')
axs[0].legend()

axs[1].plot(T2_time, T2_delta, 'g')
axs[1].plot(T2_time, qa.T2_func(T2_time, *T2_popt), 'r--', label='fit: \nampl=%5.3f \nT2=%5.3f \noffset=%5.3f' % tuple(T2_popt))
axs[1].axhline(y = Rabi_popt[3], color = 'grey', linestyle = ':')
axs[1].text(0, Rabi_popt[3], 'Rabi offset', color = 'grey', fontsize = 13)
axs[1].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'NV coherence time')
axs[1].legend()

# fig.savefig(Sample_name + " Coherence time.png")

### Fourier transform

In [ ]:
### Check qudi FFT

In [ ]:
osc = Rabi_signal
fs = Rabi_time[1] - Rabi_time[0]    # Sampling frequency, us
fft_osc = rfft(osc)        # Real values of FFT
fft_fre = rfftfreq(len(Rabi_time), d = fs)


fig, axs = plt.subplots(1, 2, figsize = (18,5))
axs[0].plot(Rabi_time, osc)
axs[0].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'Hahn Echo without exp decay')

axs[1].plot(fft_fre, np.abs(fft_osc))
axs[1].set(xlabel = 'Frequency (MHz)', ylabel = 'Amplitude (a.u.)', title = 'NV FFT')

In [ ]:
# INTERACTIVE PLOT
fft_fre = fft_fre
fft_osc = fft_osc


figexp = px.line(x = fft_fre, y = np.abs(fft_osc), title="FT")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text = "Frequency (MHz)")
figexp.update_yaxes(title_text = "Amplitude(arb. units)")
figexp.show(config = config)

# Spectrum

## Experimantal data and estimation

### Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Spectra")

filename = "Sample 19_LiF_NV12_100uW_60s.csv"
wavelength,spectrum = qa.csvReader(filename)

filename2 = "Sample 19_LiF_background_100uW_60s.csv"
wavelength2,spectrum2 = qa.csvReader(filename2)

filename3 = "Sample 19_LiF_background unbleached_100uW_60s.csv"
wavelength3,spectrum3 = qa.csvReader(filename3)
# filename4 = "LiF_5nm_scan_90 uW_60 s_90 uW.csv"
# wavelength4,spectrum4 = qa.csvReader(filename4)
# filename5 = "Sample 19_NV_60 s_90 uW.csv"
# wavelength5,spectrum5 = qa.csvReader(filename5)
# filename6 = "Jansen_PL6_30uW_60s.csv"
# wavelength6,spectrum6 = qa.csvReader(filename6)
# background1 = "Sample 17 air objective background laser 100 uW exposure time 1 min.csv"
# wavelength_b1,spectrum_b1 = qa.csvReader(background1)
# background2 = "Sample 17_background_110uW_60s.csv"
# wavelength_b2,spectrum_b2 = qa.csvReader(background2)

Removing the background.

In [ ]:
# spectrum = spectrum - spectrum_b1
# spectrum2 = spectrum2 - spectrum_b1
# spectrum3 = spectrum3 - spectrum_b2
# spectrum4 = spectrum4 - spectrum_b2

## Data analysis

### Interactive plot

In [ ]:
wavelength_x = wavelength
spectrum_y = spectrum

figexp=px.line(x=wavelength_x,y=spectrum_y,title="Fluorescence spectrum")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text="Wavelength (nm)")
figexp.update_yaxes(title_text="Counts (arb. units)")
figexp.show(config=config)

### Spectrum

In [ ]:
xmin = 550#min(wavelength)
xmax = 900 #max(wavelength)
ymin = min(spectrum)-50#min(spectrum)
ymax = max(spectrum) + 50

title = 'Sample 19 spectrum after LiF deposition'
xlabel = 'Wavelength (nm)'
ylabel = 'Counts'

In [ ]:
fig, axs = plt.subplots(figsize=(18,10))

axs.plot(wavelength, spectrum, '-', color = 'tab:blue', label='NV #12')
axs.plot(wavelength, spectrum2, '-', color = 'tab:green', label='Bleached background')
axs.plot(wavelength, spectrum3, '-', color = 'tab:orange', label='Background before bleaching')
# axs.plot(wavelength, spectrum4, '-', color = 'tab:red', label='Full area')
# axs.plot(wavelength, spectrum5, '-', color = 'tab:cyan', label='Single NV')
# axs.plot(wavelength, spectrum6, '-', color = 'tab:brown', label='Single NV')
# axs.plot(wavelength, spectrum_b, '-', color = 'tab:cyan', label='Background')

# axs.axvline(x = 650, color='grey', linestyle='--')
# axs.text(641, 650, '650 nm',color='grey',fontsize = 15)

axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
axs.grid(True)
axs.legend()
plt.show()

# fig.savefig(filename[:-4] + "_spectrum.png")

# Saturation

## Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 17") # use \\ instead of \

filename = np.loadtxt(r'SAturation encapsulated.txt', delimiter=',')
laser_power = filename[:,0] # np.array of float values of the laser power
NV_counts = filename[:,1] # np.array of float values of the intensity
NV_background = filename[:,2]# np.array of float values of the intensity
NV_sub = NV_counts - NV_background

## Data analysis

In [ ]:
saturation_popt, saturation_pcov = curve_fit(qa.saturation_func, laser_power[0:4], NV_sub[0:4])

fig, axs = plt.subplots(1,2,figsize=(18,6))

axs[0].plot(laser_power, NV_counts,'o',color='green',label='NV counts')
axs[0].plot(laser_power, NV_background,'o',color='lightgreen',label='Background counts')
axs[0].set(xlabel='Laser power (\u03bcW)', ylabel='Intensity (arb. units)', title='Single NV counts')
axs[0].legend()
axs[0].grid(True)
axs[0].ticklabel_format(axis='y',style='sci',scilimits=(5,5))
# axs[0].set(xlim=(-20, 600))

axs[1].plot(laser_power, NV_sub,'o',color='green',label='NV counts')
axs[1].plot(laser_power, NV_background,'o',color='lightgreen',label='Background counts')
axs[1].plot(laser_power, qa.saturation_func(laser_power,*saturation_popt),'g--') #, label='fit: a=%.2e' % tuple(popt)
axs[1].set(xlabel='Laser power (\u03bcW)', ylabel='Intensity (arb. units)',title='Single NV counts without background')
axs[1].legend()
axs[1].grid(True)
axs[1].ticklabel_format(axis='y', style='sci',scilimits=(5,5))
axs[1].set(xlim=(-20, 800), ylim = (-0.1,50000))

# fig.suptitle("Sample 17 (Single NV) saturation curves")

# plt.savefig("saturation enc.png")

# Lifetime

## Experimental data and estimation

### Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\SPV 18+19+ref\Lifetime")
    
filename = "Sample 19_NV0_1 min_480 nm_rep rate 2,89_current 100_LP550.mp"
lt_time, lt_counts = qa.mp_reader(filename)
lt_label1 = 'Excitation wavelength 480 nm'

filename2 = "Sample 19_NV_1 min_rep rate 2,89_current 100_LP650.mp"
lt_time2, lt_counts2 = qa.mp_reader(filename2)
lt_label2 = 'Excitation wavelength 520 nm'

### Variables for the analysis

Values to change:

lifetime: expected lifetime for the fitting.

decays: number of exponential decays for the fiting.

In [ ]:
Sample_name = 'Sample'
lifetime = 1e-8
decays = 1 # 1 or 2

## Data analysis

### Lifetime

In [ ]:
if decays == 1:
    print(lt_label1)
    time1, counts1, lt1, lt_popt1, lt_perr1 = qa.Lifetime_exp(lt_time, lt_counts, lifetime)
    print('Lifetime: ', round(lt1,3), "\u00B1", round((lt_perr1[1] * 1e9),3), 'ns')
    print('\n' + lt_label2)
    time2, counts2, lt2, lt_popt2, lt_perr2 = qa.Lifetime_exp(lt_time2, lt_counts2, lifetime)
    print('Lifetime: ', round(lt2,3), "\u00B1", round((lt_perr2[1] * 1e9),3), 'ns')
else:
    print(lt_label1)
    time1, counts1, lt1, lt_21, lt_popt1, lt_perr1 = qa.Lifetime_double_exp(lt_time, lt_counts, lifetime)
    print('Lifetime 1: ', round(lt1,3), "\u00B1", round((lt_perr1[1] * 1e9),3), 'ns')
    print('Lifetime 2: ', round(lt_21,3), "\u00B1", round((lt_perr1[3] * 1e9),3), 'ns')
    print('\n' + lt_label2)
    time2, counts2, lt2 ,lt_22, lt_popt2, lt_perr2 = qa.Lifetime_double_exp(lt_time2, lt_counts2, lifetime)
    print('Lifetime 1: ', round(lt2,3), "\u00B1", round((lt_perr2[1] * 1e9),3), 'ns')
    print('Lifetime 2: ', round(lt_22,3), "\u00B1", round((lt_perr2[3] * 1e9),3), 'ns')

### Interactive plot

In [ ]:
xdata = lt_time
ydata = lt_counts

figexp=px.line(x=xdata,y=ydata,title="Lifetime")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text="Time (ps)")
figexp.update_yaxes(title_text="Counts (arb. units)")
figexp.show(config=config)

### Lifetime plot

In [ ]:
xmin = min(time1 * 1e9)
xmax = 10 #max(time1 * 1e9)
ymin = min(counts1) - 100 #min(spectrum)
ymax = max(counts1) + 100

title = 'Sample lifetime'
xlabel = 'Time (ns)'
ylabel = 'Counts'   

In [ ]:
fig, axs = plt.subplots(figsize=(18,10))
if decays == 1:
    axs.plot(time1 * 1e9, counts1, 'o', label='Excitation wavelength 480 nm')
    axs.plot(time1 * 1e9, qa.T2_func(time1, *lt_popt1), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\noffset=%i' % tuple(lt_popt1))
    axs.plot(time2 * 1e9, counts2, 'o', label='Excitation wavelength 520 nm')
    axs.plot(time2 * 1e9, qa.T2_func(time2, *lt_popt2), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\noffset=%i' % tuple(lt_popt2))
    axs.grid(color='gray', linestyle='-', linewidth=0.1)
    axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
    axs.legend()
#     fig.savefig(filename[:-3] + "_lifetime.png")
else:
    axs.plot(time1 * 1e9, counts1, 'o', label='Excitation wavelength 480 nm')
    axs.plot(time1 * 1e9, qa.double_exp(time1, *lt_popt1), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\namplitude2=%.2e\nlifetime2=%.2e\noffset=%i' % tuple(lt_popt1))
    axs.plot(time2 * 1e9, counts2, 'o', label='Excitation wavelength 520 nm')
    axs.plot(time2 * 1e9, qa.double_exp(time2, *lt_popt2), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\namplitude2=%.2e\nlifetime2=%.2e\noffset=%i' % tuple(lt_popt2))
    axs.grid(color='gray', linestyle='-', linewidth=0.1)
    axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
    axs.legend()
#     fig.savefig(filename[:-3] + "_lifetime_double exp.png")